In [1]:
import cv2
import numpy as np
import scipy
import os
import math
import shutil
import glob
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [2]:
DATASET_PATH = "Dog vs Cat Classification"

In [3]:
number_of_images = {}

for dir in os.listdir(DATASET_PATH):
    number_of_images[dir] = len(os.listdir(os.path.join(DATASET_PATH, dir)))

In [4]:
number_of_images

{'Cat': 5011, 'Dog': 5017}

In [5]:
def data_folder(folder_name, split_ratio):
    if not os.path.exists("./" + folder_name):
        os.mkdir("./" + folder_name)
    
    for dir in os.listdir(DATASET_PATH):
        if not os.path.exists("./" + folder_name + "/" + dir):
            os.makedirs("./" + folder_name + "/" + dir)

        for img in np.random.choice(a = os.listdir(os.path.join(DATASET_PATH, dir)), size = (math.floor(split_ratio * number_of_images[dir])), replace = False):
            O = os.path.join(DATASET_PATH, dir, img)
            D = os.path.join("./" + folder_name, dir)
            shutil.copy(O, D)
    
    else:
        print(f"{folder_name} Folder exist")

In [6]:
data_folder("Train", 0.7)

Train Folder exist


In [7]:
data_folder("Validate", 0.15)

Validate Folder exist


In [8]:
data_folder("Test", 0.15)

Test Folder exist


In [9]:
TRAIN_PATH = "./Train"

In [10]:
TEST_PATH = "./Test"

In [11]:
VALIDATE_PATH = "./Validate"

In [12]:
class_names_train = os.listdir(TRAIN_PATH)

In [13]:
class_names_test = os.listdir(TEST_PATH)

In [14]:
class_names_validate = os.listdir(VALIDATE_PATH)

In [15]:
print(class_names_train)

['Cat', 'Dog']


In [16]:
print(class_names_test)

['Cat', 'Dog']


In [17]:
print(class_names_validate)

['Cat', 'Dog']


In [18]:
train_datagen = ImageDataGenerator(zoom_range = 0.15, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.15)
validate_datagen = ImageDataGenerator()

In [19]:
train_generator = train_datagen.flow_from_directory("./Train", target_size = (224, 224), batch_size = 32, shuffle=True, class_mode = 'binary')
validate_generator = validate_datagen.flow_from_directory("./Validate", target_size = (224,224), batch_size = 32, shuffle = False, class_mode='binary')

Found 7018 images belonging to 2 classes.
Found 1503 images belonging to 2 classes.


In [20]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut]) # SKIP Connection
    X = Activation('relu')(X)

    return X

In [21]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [22]:
def ResNet50(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [23]:
base_model = ResNet50(input_shape=(224, 224, 3))

In [24]:
print(base_model)

In [25]:
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( 1,activation='sigmoid', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)

In [26]:
model = Model(inputs=base_model.input, outputs=headModel)

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

                                                                                                  
 activation_8 (Activation)      (None, 55, 55, 64)   0           ['bn2c_branch2b[0][0]']          
                                                                                                  
 res2c_branch2c (Conv2D)        (None, 55, 55, 256)  16640       ['activation_8[0][0]']           
                                                                                                  
 bn2c_branch2c (BatchNormalizat  (None, 55, 55, 256)  1024       ['res2c_branch2c[0][0]']         
 ion)                                                                                             
                                                                                                  
 add_2 (Add)                    (None, 55, 55, 256)  0           ['bn2c_branch2c[0][0]',          
                                                                  'activation_6[0][0]']           
          

 ion)                                                                                             
                                                                                                  
 add_5 (Add)                    (None, 28, 28, 512)  0           ['bn3c_branch2c[0][0]',          
                                                                  'activation_15[0][0]']          
                                                                                                  
 activation_18 (Activation)     (None, 28, 28, 512)  0           ['add_5[0][0]']                  
                                                                                                  
 res3d_branch2a (Conv2D)        (None, 28, 28, 128)  65664       ['activation_18[0][0]']          
                                                                                                  
 bn3d_branch2a (BatchNormalizat  (None, 28, 28, 128)  512        ['res3d_branch2a[0][0]']         
 ion)     

 add_8 (Add)                    (None, 14, 14, 1024  0           ['bn4b_branch2c[0][0]',          
                                )                                 'activation_24[0][0]']          
                                                                                                  
 activation_27 (Activation)     (None, 14, 14, 1024  0           ['add_8[0][0]']                  
                                )                                                                 
                                                                                                  
 res4c_branch2a (Conv2D)        (None, 14, 14, 256)  262400      ['activation_27[0][0]']          
                                                                                                  
 bn4c_branch2a (BatchNormalizat  (None, 14, 14, 256)  1024       ['res4c_branch2a[0][0]']         
 ion)                                                                                             
          

                                                                                                  
 res4f_branch2a (Conv2D)        (None, 14, 14, 256)  262400      ['activation_36[0][0]']          
                                                                                                  
 bn4f_branch2a (BatchNormalizat  (None, 14, 14, 256)  1024       ['res4f_branch2a[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_37 (Activation)     (None, 14, 14, 256)  0           ['bn4f_branch2a[0][0]']          
                                                                                                  
 res4f_branch2b (Conv2D)        (None, 14, 14, 256)  590080      ['activation_37[0][0]']          
                                                                                                  
 bn4f_bran

 ion)                                                                                             
                                                                                                  
 activation_46 (Activation)     (None, 7, 7, 512)    0           ['bn5c_branch2a[0][0]']          
                                                                                                  
 res5c_branch2b (Conv2D)        (None, 7, 7, 512)    2359808     ['activation_46[0][0]']          
                                                                                                  
 bn5c_branch2b (BatchNormalizat  (None, 7, 7, 512)   2048        ['res5c_branch2b[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_47 (Activation)     (None, 7, 7, 512)    0           ['bn5c_branch2b[0][0]']          
          

In [28]:
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [29]:
base_model.load_weights("./resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [30]:
for layer in base_model.layers:
    layer.trainable = False

In [31]:
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x0000020D5343DFC0> False
<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x0000020D5343D4E0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x0000020D5343E9B0> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020D5343EA40> False
<keras.layers.core.activation.Activation object at 0x0000020D50D13C70> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x0000020D5343D900> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x0000020D532BC970> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020D532BF040> False
<keras.layers.core.activation.Activation object at 0x0000020D532BF280> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x0000020D532BC040> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020D532BC130> False
<keras.layers.core.activation.Activation object at 0x000

In [32]:
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

In [33]:
mc = ModelCheckpoint(filepath='./best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)

In [ ]:
H = model.fit_generator(generator=train_generator, validation_data=validate_generator, epochs=100, verbose=1, callbacks=[mc,es])

C:\Users\hamza\AppData\Local\Temp\ipykernel_14292\2258209806.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(generator=train_generator, validation_data=validate_generator, epochs=100, verbose=1, callbacks=[mc,es])


Epoch 1/100
136/220 [=================>............] - ETA: 30:01 - loss: 0.6449 - accuracy: 0.7843

In [ ]:
model.load_weights("./best_model.h5")

In [ ]:
model.evaluate_generator(validate_generator)

In [ ]:
model_json = model.to_json()
with open("./model.json","w") as json_file:
    json_file.write(model_json)

In [ ]:
from keras.models import model_from_json

In [ ]:
def predict_(image_path):
    #Load the Model from Json File
    json_file = open('./model.json', 'r')
    model_json_c = json_file.read()
    json_file.close()
    model_c = model_from_json(model_json_c)
    #Load the weights
    model_c.load_weights("./best_model.h5")
    #Compile the model
    opt = SGD(lr=1e-4, momentum=0.9)
    model_c.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    #load the image you want to classify
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224,224))
    cv2_imshow(image)
    #predict the image
    preds = model_c.predict(np.expand_dims(image, axis=0))[0]
    if preds==0:
        print("Predicted Label:Cat")
    else:
        print("Predicted Label: Dog")

In [ ]:
predict_("./Test/Dog/4.jpg")

In [ ]:
predict_("./Test/Cat/10.jpg")